In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer
import pyspark.pandas as ps


os.environ['PYSPARK_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\python.exe'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'C:\ProgramData\mambaforge\envs\ML-base\Scripts\ipython.exe'

spark = SparkSession.builder.appName("MyApp").getOrCreate()
sc=spark.sparkContext
spark

c:\ProgramData\mambaforge\envs\ML-base\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.ml.linalg import SparseVector
from scipy.sparse import csr_matrix
from scipy.sparse import rand
import numpy as np


# Create a sample SciPy CSR matrix
rows, cols = 5, 10
density = 0.2
rng = np.random.default_rng()
csr = rand(rows, cols, density=density, format="csr", random_state=rng)

# Convert the SciPy CSR matrix to an RDD of SparseVectors
def csr_to_sparse_vector(csr_row):
    return SparseVector(csr_row.shape[1], list(zip(csr_row.indices, csr_row.data)))

rdd_sv = spark.sparkContext.parallelize([csr_to_sparse_vector(row) for row in csr])

#df = spark.createDataFrame(rdd.map(lambda x: (x,)), schema=schema)

# Show the DataFrame


In [5]:
rdd_sv.collect()

[SparseVector(10, {0: 0.0633, 2: 0.8776, 9: 0.3118}),
 SparseVector(10, {7: 0.2585}),
 SparseVector(10, {7: 0.8844}),
 SparseVector(10, {7: 0.4276, 8: 0.6353}),
 SparseVector(10, {0: 0.7499, 3: 0.768, 7: 0.5756})]

In [ ]:
dataset="datasets/nfcorpus/corpus.jsonl"

In [ ]:
df_docs=spark.read.json(dataset).select('_id','text')

In [ ]:
#TODO: check lower, stemming etc
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df_docs)

tf = CountVectorizer(inputCol="words", outputCol="rawFeatures")
tf_data = tf.fit(wordsData).transform(wordsData)

idf = IDF(inputCol="rawFeatures", outputCol="features")
tf_idf_data = idf.fit(tf_data).transform(tf_data).select("_id", "features")

In [ ]:
tf_idf_data.rdd

In [ ]:
tf_idf_data.select("features").first()[0][2]

In [ ]:
n_acc=tf_idf_data.select("features").first()[0].size

In [ ]:
acc = [0.0 for i in range(n_acc)]

In [ ]:
def custom_max(vect):
    global acc
    for i in range(vect[0].size):
        if vect[0][i]>acc[i]:
            acc[i]=vect[0][i]

In [ ]:
tf_idf_data.select("features").rdd.foreach(custom_max)

In [ ]:

spark.stop()

In [ ]:
""" n_acc=tf_idf_data.select("features").first()[0].size
max_acc =  sc.[0.0 for i in range(n_acc)]

# merge by summing partial_sum and adding 1 to partial_count
def mergeValue(acc, vect):
    for i in range(vect[0].size):
        if vect[0][i]>acc[i]:
            acc[i]=vect[0][i]
    return acc
# merge by summing partial_sums and partial_counts
def mergeAccum(acc1, acc2): return [acc1[i] if acc1[i]>acc2[i] else acc2[i] for i in range(len(acc1))]
# spark aggregate
d_star = tf_idf_data.select("features").rdd.aggregate(max_acc, mergeValue, mergeAccum) """